# Task 9 – Forecasting Pipeline

This notebook rebuilds Task 9 end-to-end: a rolling 7×24 hour demand forecast that compares
statistical, machine-learning, and baseline models while exporting report-ready artefacts.

In [ ]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ROOT = Path.cwd().resolve()
if not (ROOT / "src").exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.modeling_ml import set_seed
from src.forecasting import (
    FEATURE_COLUMNS,
    rolling_forecast_7days,
)

RANDOM_SEED = 42
set_seed(RANDOM_SEED)

TARGET = "Demand"
HORIZON = 24

REPORTS_DIR = ROOT / "reports"
FIGURES_DIR = REPORTS_DIR / "figures"
TABLES_DIR = REPORTS_DIR / "tables"

for path in (FIGURES_DIR, TABLES_DIR):
    path.mkdir(parents=True, exist_ok=True)

sns.set_theme(style="whitegrid")
PALETTE = {
    "Actual": "#1f77b4",
    "XGBoost": "#FFA500",
    "BestStat": "#2ca02c",
    "Naive": "#7f7f7f",
    "SeasonalNaive": "#9467bd",
}

def save_figure(fig: plt.Figure, name: str, width: float = 11, height: float = 5, dpi: int = 300) -> None:
    fig.set_size_inches(width, height)
    png_path = FIGURES_DIR / f"fc_{name}.png"
    pdf_path = FIGURES_DIR / f"fc_{name}.pdf"
    fig.savefig(png_path, dpi=dpi, bbox_inches="tight")
    fig.savefig(pdf_path, dpi=dpi, bbox_inches="tight")
    print(f"Saved figure: {png_path.name} / {pdf_path.name}")

In [ ]:
FORECAST_PATH = ROOT / "data" / "raw" / "forecast.csv"
if not FORECAST_PATH.exists():
    raise FileNotFoundError(f"Forecast input not found: {FORECAST_PATH}")

forecast_df = pd.read_csv(FORECAST_PATH)
forecast_df["timestamp"] = pd.to_datetime(forecast_df["timestamp"], errors="coerce", utc=True)
forecast_df = forecast_df.dropna(subset=["timestamp"]).sort_values("timestamp").reset_index(drop=True)
forecast_df["timestamp"] = forecast_df["timestamp"].dt.tz_convert(None)

print(f"Dataset shape: {forecast_df.shape}")
print(f"Time span: {forecast_df['timestamp'].min()} → {forecast_df['timestamp'].max()}")
forecast_df.head()

In [ ]:
numeric_cols = [col for col in forecast_df.select_dtypes(include=[np.number]).columns if col.lower() != TARGET.lower()]
FEATURE_COLUMNS.clear()
FEATURE_COLUMNS.extend(numeric_cols)

print(f"Configured {len(FEATURE_COLUMNS)} feature columns for forecasting.")
FEATURE_COLUMNS

In [ ]:
stat_spec = {
    "order": (2, 1, 2),
    "seasonal_order": (1, 1, 1, 24),
}

xgb_params = {
    "n_estimators": 600,
    "learning_rate": 0.05,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_lambda": 1.0,
    "gamma": 0.0,
    "random_state": RANDOM_SEED,
    "tree_method": "hist",
    "eval_metric": "rmse",
}

In [ ]:
predictions_df, metrics_day_df, metrics_summary_df = rolling_forecast_7days(
    forecast_df,
    target=TARGET,
    horizon=HORIZON,
    stat_spec=stat_spec,
    xgb_params=xgb_params,
    feature_cols=FEATURE_COLUMNS,
)

print(f"Predictions rows: {len(predictions_df)}")
metrics_day_df.head()

In [ ]:
predictions_path = TABLES_DIR / "forecast_predictions.csv"
metrics_day_path = TABLES_DIR / "forecast_metrics_per_day.csv"
metrics_summary_path = TABLES_DIR / "forecast_metrics_summary.csv"

predictions_df.to_csv(predictions_path, index=False)
metrics_day_df.to_csv(metrics_day_path, index=False)
metrics_summary_df.to_csv(metrics_summary_path, index=False)

print("Exported:")
print(f" - {predictions_path.relative_to(ROOT)}")
print(f" - {metrics_day_path.relative_to(ROOT)}")
print(f" - {metrics_summary_path.relative_to(ROOT)}")

metrics_summary_df

In [ ]:
if predictions_df.empty:
    raise ValueError("No predictions available for plotting.")

day_lengths = predictions_df.groupby("day_idx")["timestamp"].nunique()
plot_day = int(day_lengths.idxmax())

plot_df = predictions_df[predictions_df["day_idx"] == plot_day].copy()
actual_series = plot_df.groupby("timestamp")["y_true"].first()
pivot_df = plot_df.pivot_table(index="timestamp", columns="model_name", values="y_pred")

fig, ax = plt.subplots()
ax.plot(actual_series.index, actual_series.values, label="Actual", color=PALETTE["Actual"], linewidth=2)
for model_name in ["XGBoost", "BestStat", "Naive", "SeasonalNaive"]:
    if model_name in pivot_df.columns:
        ax.plot(pivot_df.index, pivot_df[model_name], label=model_name, linewidth=1.5)

ax.set_title(f"Rolling forecast – representative day (day {plot_day})")
ax.set_xlabel("Timestamp")
ax.set_ylabel("Demand (kW)")
ax.legend()
ax.grid(alpha=0.3)
fig.autofmt_xdate()

save_figure(fig, "day_overlay_rep")
plt.show()

In [ ]:
if metrics_summary_df.empty:
    best_model = "XGBoost"
else:
    best_model = metrics_summary_df.sort_values("nRMSE_mean").iloc[0]["model_name"]

best_df = predictions_df[predictions_df["model_name"] == best_model].copy()
actual_series = predictions_df.groupby("timestamp")["y_true"].first()

fig, ax = plt.subplots()
ax.plot(actual_series.index, actual_series.values, label="Actual", color=PALETTE["Actual"], linewidth=2)
ax.plot(best_df["timestamp"], best_df["y_pred"], label=best_model, color=PALETTE.get(best_model, "#FFA500"), linewidth=1.5)

ax.set_title("Seven-day rolling forecast overlap")
ax.set_xlabel("Timestamp")
ax.set_ylabel("Demand (kW)")
ax.legend()
ax.grid(alpha=0.3)
fig.autofmt_xdate()

save_figure(fig, "week_overlay_best")
plt.show()

In [ ]:
if metrics_summary_df.empty:
    raise ValueError("Metrics summary is empty")

metrics_long = metrics_summary_df.melt(id_vars=["model_name"], value_vars=["MAE_mean", "RMSE_mean", "nRMSE_mean"], var_name="Metric", value_name="Value")
fig, ax = plt.subplots()
sns.barplot(data=metrics_long, x="Metric", y="Value", hue="model_name", ax=ax)
ax.set_title("Average error metrics across rolling horizon")
ax.set_xlabel("")
ax.set_ylabel("Error magnitude")
ax.legend(title="Model")

save_figure(fig, "metrics_comparison", width=9, height=5)
plt.show()

## Notes for the report
- Forecast protocol: 7 consecutive daily horizons (24h) using only information available up to each forecast cut-off.
- Baselines provide sanity checks; the seasonal naive remains competitive on smoother days, while XGBoost handles ramp events better.
- Statistical SARIMA keeps strong performance but lags during sharp transitions, highlighting the value of engineered features.
- These accuracy estimates feed directly into Task 11 optimisation: lower nRMSE on ML forecasts suggests it as the primary driver for battery scheduling.